In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
with open("news//news_train.txt", encoding='utf-8') as f:
    train  = f.readlines()
with open("news//news_test.txt", encoding='utf-8') as f:
    test  = f.readlines()

In [3]:
import random
random.shuffle(train)

In [4]:
from collections import Counter

Counter([line.split('\t')[0] for line in train[:15000]])

Counter({'science': 2156,
         'forces': 1225,
         'culture': 2053,
         'sport': 2215,
         'media': 2111,
         'life': 2033,
         'economics': 2080,
         'business': 554,
         'style': 284,
         'travel': 289})

In [5]:
import re
def tokenize(lines, regex_string=r'\b\w+\b'):
    result = []
    for line in lines:
        label, title, text = line.split('\t')
        
        title = re.findall(regex_string, title.lower())
        text = re.findall(regex_string, text.lower())
        
        result.append(dict())
        result[-1]['label'] = label
        result[-1]['title'] = title
        result[-1]['text'] = text
        
    return result

In [6]:
train_data = tokenize(train)
train_data[0]

{'label': 'science',
 'title': ['британцы', 'секвенируют', 'геном', 'ричарда', 'iii'],
 'text': ['британские',
  'ученые',
  'секвенируют',
  'геном',
  'ричарда',
  'iii',
  'об',
  'этом',
  'сообщает',
  'the',
  'guardian',
  'секвенированием',
  'называется',
  'процесс',
  'определения',
  'аминокислотной',
  'или',
  'нуклеотидной',
  'последовательности',
  'молекулы',
  'в',
  'данном',
  'случае',
  'днк',
  'по',
  'данным',
  'издания',
  'стоимость',
  'проекта',
  'составляет',
  '100',
  'тысяч',
  'фунтов',
  'ученые',
  'признают',
  'что',
  'чтение',
  'генома',
  'ричарда',
  'iii',
  'представляет',
  'определенные',
  'трудности',
  'днк',
  'приходится',
  'извлекать',
  'из',
  'костей',
  'которые',
  'провели',
  'в',
  'земле',
  'около',
  '500',
  'лет',
  'король',
  'был',
  'похоронен',
  'без',
  'гроба',
  'исследователи',
  'говорят',
  'что',
  'их',
  'целью',
  'является',
  'поиск',
  'маркеров',
  'которые',
  'могли',
  'бы',
  'уточнить',
  'вн

In [7]:
import pymorphy2
from tqdm.notebook import tqdm
def lemmatize(lines):
    morph_analyzer = pymorphy2.MorphAnalyzer()
    for line in tqdm(lines):
        line['title'] = list(map(lambda x: morph_analyzer.parse(x)[0].normal_form, line['title']))
        line['text'] = list(map(lambda x: morph_analyzer.parse(x)[0].normal_form, line['text']))

In [8]:
lemmatize(train_data)
train_data[0]

{'label': 'science',
 'title': ['британец', 'секвенировать', 'геном', 'ричард', 'iii'],
 'text': ['британский',
  'учёный',
  'секвенировать',
  'геном',
  'ричард',
  'iii',
  'о',
  'это',
  'сообщать',
  'the',
  'guardian',
  'секвенирование',
  'называться',
  'процесс',
  'определение',
  'аминокислотный',
  'или',
  'нуклеотидный',
  'последовательность',
  'молекула',
  'в',
  'дать',
  'случай',
  'днк',
  'по',
  'данные',
  'издание',
  'стоимость',
  'проект',
  'составлять',
  '100',
  'тысяча',
  'фунт',
  'учёный',
  'признавать',
  'что',
  'чтение',
  'геном',
  'ричард',
  'iii',
  'представлять',
  'определённый',
  'трудность',
  'днк',
  'приходиться',
  'извлекать',
  'из',
  'кость',
  'который',
  'провести',
  'в',
  'земля',
  'около',
  '500',
  'год',
  'король',
  'быть',
  'похоронный',
  'без',
  'гроб',
  'исследователь',
  'говорят',
  'что',
  'они',
  'цель',
  'являться',
  'поиск',
  'маркер',
  'который',
  'мочь',
  'бы',
  'уточнить',
  'внешний'

## Word embeddings

In [9]:
import pandas as pd
train_df = pd.DataFrame(train_data)
train_df.head()

,label,title,text
0,science,"[британец, секвенировать, геном, ричард, iii]","[британский, учёный, секвенировать, геном, рич..."
1,forces,"[в, москва, задержать, сын, замглавы, лукойл]","[cын, вица, президент, компания, лукойл, русла..."
2,forces,"[новый, фигурант, болотный, дело, отправить, п...","[басманный, суд, москва, не, удовлетворить, хо..."
3,culture,"[мэрил, стрип, предложить, возглавить, идеальн...","[мэрил, стрип, предложить, роль, в, фильм, дав..."
4,sport,"[эксперт, сообщить, о, положительный, допинг, ...","[российский, боксёр, супертяжелый, весовой, ка..."


In [10]:
from gensim.models import Word2Vec
word2vec = Word2Vec(train_df['text'].append(train_df['title']))

words = [ 'Привет' , 'футбол' , 'москва', 'машина' , 'олимпиада']
for word in words:
    print(word2vec.wv.most_similar(positive=[word.lower()]),'\n')

[('вулф', 0.8315575122833252), ('ха', 0.8252878785133362), ('бутырский', 0.8201467990875244), ('уилска', 0.8191892504692078), ('камбоджийский', 0.817003071308136), ('мохнаткин', 0.8169586062431335), ('циркач', 0.8144558668136597), ('терьер', 0.8128005266189575), ('слушаться', 0.8124203681945801), ('царнаева', 0.8119717836380005)] 

[('хоккей', 0.8957664966583252), ('теннис', 0.841028094291687), ('баскетбол', 0.7802850008010864), ('биатлон', 0.7438477873802185), ('фигурный', 0.7256494164466858), ('сборный', 0.7244349718093872), ('шахматы', 0.7181534767150879), ('сборная', 0.714941680431366), ('атлетика', 0.7119095325469971), ('первенство', 0.7014695405960083)] 

[('петербург', 0.7038277387619019), ('минск', 0.6358433961868286), ('крым', 0.6222398281097412), ('санкт', 0.6128217577934265), ('новосибирск', 0.6070311069488525), ('киев', 0.5775637626647949), ('столица', 0.5736709833145142), ('владивосток', 0.5727264881134033), ('столичный', 0.572583794593811), ('московский', 0.56688338518142

## Классификация

In [11]:
labels_dict  = {train_df.label.unique()[i] : i for i in range(train_df.label.nunique())}
labels_dict 

{'science': 0,
 'forces': 1,
 'culture': 2,
 'sport': 3,
 'media': 4,
 'life': 5,
 'economics': 6,
 'business': 7,
 'style': 8,
 'travel': 9}

In [12]:
def map_labels(labels, labels_dict):
    return labels.map(lambda label: labels_dict[label]).to_list()

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np

def mean_tfidf_vectorizer(X, word2vec):
    word2vec_dict = dict(zip(word2vec.wv.index2word, word2vec.wv.vectors))
    dim = len(next(iter(word2vec_dict.values())))        
    
    tfidf = TfidfVectorizer(analyzer=lambda x: x)
    tfidf.fit(X)
    max_idf = max(tfidf.idf_)
    word2weight = defaultdict(
        lambda: max_idf, 
        [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
    return np.array([
        np.mean([word2vec[w] * word2weight[w] 
                 for w in words if w in word2vec] or 
                [np.zeros(dim)], axis=0) 
        for words in X
    ])

In [14]:
x_train = mean_tfidf_vectorizer(train_df['text'], word2vec)
y_train = map_labels(train_df['label'], labels_dict)

In [15]:
test_data = tokenize(test)

lemmatize(test_data)
test_df = pd.DataFrame(test_data)

x_test = mean_tfidf_vectorizer(test_df['text'], word2vec)
y_test = map_labels(test_df['label'], labels_dict)

In [16]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression(max_iter=100)
clf.fit(x_train, y_train);
predictions = clf.predict(x_test)
print(f"Accuracy score {accuracy_score(y_test, predictions)}")

Accuracy score 0.8393333333333334


In [17]:
clf = SVC()
clf.fit(x_train, y_train);
predictions = clf.predict(x_test)
print(f"Accuracy score {accuracy_score(y_test, predictions)}")

Accuracy score 0.8513333333333334


In [18]:
clf = GaussianNB()
clf.fit(x_train, y_train);
predictions = clf.predict(x_test)
print(f"Accuracy score {accuracy_score(y_test, predictions)}")

Accuracy score 0.7216666666666667
